In the last notebook, we worked with chunking due to the token limit with large language models. If there are gigabytes of data, we will have lots of chunks to be created as well. Is there a way to select the most relevant chunks of text? The answer is yes. To solve this problem, we can take a look at a process called Embedding. Embedding helps us create numerical representations for all the chunks. Then, we can find the most similar chunks in the the list of embeddings. One popular way to find the similar chunks is through cosine similarity.

### **Embeddings Overview**
An embedding is a special format of data representation that can be easily utilized by machine learning models and algorithms. The embedding is an information dense representation of the semantic meaning of a piece of text. Each embedding is a vector of floating-point numbers, such that the distance between two embeddings in the vector space is correlated with semantic similarity between two inputs in the original format. For example, if two texts are similar, then their vector representations should also be similar.

Different Azure OpenAI embedding models are specifically created to be good at a particular task. Similarity embeddings are good at capturing semantic similarity between two or more pieces of text. Text search embeddings help measure long documents are relevant to a short query. Code search embeddings are useful for embedding code snippets and embedding nature language search queries.

Embeddings make it easier to do machine learning on large inputs representing words by capturing the semantic similarities in a vector space. Therefore, we can use embeddings to if two text chunks are semantically related or similar, and inherently provide a score to assess similarity.

### **Cosine Similarity**
A previously used approach to match similar documents was based on counting maximum number of common words between documents. This is flawed since as the document size increases, the overlap of common words increases even if the topics differ. Therefore cosine similarity is a better approach.

Mathematically, cosine similarity measures the cosine of the angle between two vectors projected in a multi-dimensional space. This is beneficial because if two documents are far apart by Euclidean distance because of size, they could still have a smaller angle between them and therefore higher cosine similarity.

The Azure OpenAI embeddings rely on cosine similarity to compute similarity between documents and a query.

### **Applications**

Embeddings can be created for all different data types including images, audio, video, and text. In this notebook, we will look at generating embeddings for text and csv files. 

There are many applications in which embeddings can be useful. For example, let's say you want to classify a piece of text. Once embeddings are generated, they can be inserted into a machine learning model to predict the right label. A very popular use case is one that involves semantic search. If you want to retrieve documents that are very relevant to your query, embeddings can be generated for both the query as well as the documents in order to get an accurate response. We will see an example of this in Challenge 4.

In [ ]:
import openai
import os
import re 
import requests
import sys
from num2words import num2words 
import pandas as pd 
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity 
import tiktoken
from dotenv import load_dotenv
from tenacity import retry, wait_random_exponential, stop_after_attempt
load_dotenv() 
openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_key = os.environ.get("OPENAI_API_KEY")
openai.api_base = os.environ.get("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
embedding_model=os.getenv("EMBEDDING_MODEL_NAME")

Generate Embeddings on text

In [ ]:
embedding = openai.Embedding.create(
    input="I would like to order a pizza", engine=embedding_model
)["data"][0]["embedding"]
print(embedding)
len(embedding)

Here, we are using the Embedding class to create an embedding for the piece of text. The openai.Embedding.create() method will take a list of text - here we have a single sentence - and then will return a list containing a single embedding. You can use these embeddings when searching, providing recommendations, classification, and more.

Generate Embeddings for a CSV file

In [ ]:
df=pd.read_csv(os.path.join(os.getcwd(),r'C:\Users\dthakar\Documents\WhatTheHack\xxx-OpenAIFundamentals\Student\Resources\data\Automobile.csv'))
df

In [ ]:
shortened_df = df[['name', 'mpg', 'origin']]
shortened_df

In [ ]:
tokenizer = tiktoken.get_encoding("cl100k_base")
shortened_df['n_tokens'] = shortened_df["name"].apply(lambda x: len(tokenizer.encode(x)))
shortened_df = shortened_df[shortened_df.n_tokens<8192]
len(shortened_df)

In [ ]:
shortened_df

In [ ]:
sample_encode = tokenizer.encode(shortened_df.name[0]) 
decode = tokenizer.decode_tokens_bytes(sample_encode)
decode

In [ ]:
len(decode)

In [ ]:
shortened_df['ada_v2'] = shortened_df["name"].apply(lambda x : get_embedding(x, engine = 'text-embedding-ada-002')) # engine should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model

In [ ]:
shortened_df

These embeddings generated from the csv file can be used to perform search. You can calculate the cosine similarity between a query embedding and the embeddings from the csv file. Then you can rank the search results to what is most relevant to the query. We will see an application of embedddings in the next challenge.